# The Dynamic Impact of Foreign Direct Investment on Economic Growth in BRICS Countries

## Introduction
Foreign Direct Investment (FDI) is a crucial catalyst for global economic integration, particularly for emerging economies. The BRICS nations (Brazil, Russia, India, China, South Africa) have become significant FDI recipients, yet the precise impact of FDI on their economic growth remains a subject of debate, showing varied patterns across member states.

This research will rigorously investigate the dynamic and multifaceted relationship between FDI and economic growth in BRICS countries from 1995 to 2023. Employing advanced panel data methodologies, this study will explore FDI's long-run contribution to GDP, the mechanisms through which this occurs, and how critical macroeconomic factors (e.g., financial development, trade openness, gross fixed capital formation, and inflation) condition this relationship.

## Research Questions

**Main Research Question**  
What is the dynamic and conditional impact of Foreign Direct Investment on economic growth in BRICS countries, considering macroeconomic factors?

**Sub-Questions**
- What is the magnitude and persistence of FDI's influence on economic growth in BRICS nations?
- How do crucial macroeconomic factors (e.g., Gross Fixed Capital Formation, Trade, Inflation, Financial Development) affect the FDI-growth nexus?
- Has the relationship between FDI and economic growth in BRICS countries undergone significant changes following major global events, such as the 2008 Global Financial Crisis?


## Import packages

In [57]:
# Install required packages (Colab only)
!pip install linearmodels arch openpyxl --quiet

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from linearmodels.panel import PanelOLS, RandomEffects
from statsmodels.formula.api import ols
from arch.unitroot import ADF
from scipy import stats
from google.colab import files
from statsmodels.tsa.stattools import adfuller




## Upload and Read Data

In [5]:
# Read csv Data and convert it into a pandas DataFrame
df = pd.read_csv('BRICS_data.csv')
df = pd.DataFrame(df)

# Investigate shape of dataset, datatypes, column names
print("Shape of dataset:", df.shape)
print("\nData types:")
print(df.dtypes)
print("\nOriginal columns:")
print(df.columns.tolist())

Shape of dataset: (150, 9)

Data types:
Unnamed: 0                                             int64
Country Name                                          object
Year                                                   int64
Domestic credit to private sector (% of GDP)         float64
Foreign direct investment, net inflows (% of GDP)    float64
GDP per capita (constant 2015 US$)                   float64
Gross fixed capital formation (% of GDP)             float64
Inflation, consumer prices (annual %)                float64
Trade (% of GDP)                                     float64
dtype: object

Original columns:
['Unnamed: 0', 'Country Name', 'Year', 'Domestic credit to private sector (% of GDP)', 'Foreign direct investment, net inflows (% of GDP)', 'GDP per capita (constant 2015 US$)', 'Gross fixed capital formation (% of GDP)', 'Inflation, consumer prices (annual %)', 'Trade (% of GDP)']


## Preliminary Data Exploration and Data Cleaning


In [27]:
# Map long names to short ones used in the script:
rename_map = {
    'Country Name': 'country',
    'Year': 'year',
    'Foreign direct investment, net inflows (% of GDP)': 'fdi',
    'GDP per capita (constant 2015 US$)': 'gdp_pc',      # will be used to compute growth
    'Gross fixed capital formation (% of GDP)': 'gfcf',
    'Inflation, consumer prices (annual %)': 'inflation',
    'Trade (% of GDP)': 'trade',
    'Domestic credit to private sector (% of GDP)': 'domcred'
}

df = df.rename(columns=rename_map)

# Keep only relevant columns (if extra columns exist they are ignored)
keep_cols = ['country', 'year', 'gdp_pc', 'fdi', 'gfcf', 'inflation', 'trade', 'domcred']
df = df[[c for c in keep_cols if c in df.columns]]

# -------------------------
# 3) Type conversions & cleaning
# -------------------------
# Ensure year is integer and country is string
df['year'] = df['year'].astype(int)
df['country'] = df['country'].astype(str)

# Convert numeric-like columns to numeric (handles strings like "1,234" etc.)
num_cols = ['gdp_pc', 'fdi', 'gfcf', 'inflation', 'trade', 'domcred']
for col in num_cols:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')

# Interpolate numeric missing values within each country (time series)
df = df.sort_values(['country', 'year'])
df[num_cols] = df.groupby('country')[num_cols].transform(lambda g: g.interpolate(method='linear', limit_direction='both'))

# Investigate first few rows
df.head(10)


,country,year,gdp_pc,fdi,gfcf,inflation,trade,domcred
0,Brazil,1995,6596.335727,0.631586,20.286298,66.007034,16.984460,43.494525
1,Brazil,1996,6640.727007,1.475965,18.640654,15.757666,15.635591,40.778494
2,Brazil,1997,6764.858421,2.150453,19.122901,6.926713,16.576209,40.852237
3,Brazil,1998,6687.495236,3.340888,18.542349,3.195076,16.438585,29.532261
4,Brazil,1999,6621.636762,4.733770,17.016294,4.858447,20.982166,29.826910
5,Brazil,2000,6817.784241,5.033917,18.304488,7.044141,22.639761,31.140986
6,Brazil,2001,6823.033970,4.147594,18.418087,6.840359,26.936285,29.004038
7,Brazil,2002,6944.623375,3.253581,17.926251,8.450164,27.618357,29.645142
8,Brazil,2003,6941.440457,1.813401,16.604759,14.714920,28.140385,27.685670
9,Brazil,2004,7258.781852,2.713532,17.320233,6.597185,29.678252,29.372770


## Adding GDP per capita to the dataframe

In [40]:
# Read the CSV file containing gdp per capita growth data
df2 = pd.read_csv('gdp_per_capita_growth.csv')

# Filter the dataframe to include only BRICS countries
df2 = df2[df2['Country Name'].isin(['Brazil', 'India', 'China', 'Russia', 'South Africa'])]

# Drop unwanted columns
df2.drop(columns=['Country Code', 'Series Name', 'Series Code'], inplace=True)

# Convert the dataframe to long format
df2 = df2.melt(id_vars=['Country Name'], var_name='Year', value_name='gdp_pc_growth')
df2.rename(columns={'Country Name': 'country'}, inplace=True)
df2['year'] = df2['Year'].astype(str).str[:4].astype(int)

# Sort the dataframe according to the country then year
df2.sort_values(by=['country', 'year'], inplace=True)
df2.reset_index(drop=True, inplace=True)

# Filter the data to include only rows containing data after 1994
df2 = df2[df2['year'] > 1994]
df2.drop(columns=['Year'], inplace=True)
# print(df2.head(5))

# Merge the data on country and year
df_merged = pd.merge(df, df2, on=['country', 'year'], how='left')
df_merged.head(10)

df_merged[df_merged['gdp_pc_growth'].isna()].shape

(34, 9)

# Calculating GDP per capita growth from GDP per capita

In [41]:
# Compute annual percent change of GDP per capita (multiply by 100 for percent)
df['gdp_growth'] = df.groupby('country')['gdp_pc'].transform(lambda x: x.pct_change() * 100)

# Verify the dataset

In [52]:
# Filtering the dataset to include data from 2000 to 2023
df_final = df[(df['year'] >= 2000) & (df['year'] <= 2023)]

print("\nAfter cleaning, sample rows:")
display(df.head())

print("\nData types:")
print(df.dtypes)

missing_rows = df_final[df_final['gdp_growth'].isna()].shape[0]
print(f"\nNumber of missing rows: {missing_rows}")


After cleaning, sample rows:


,country,year,gdp_pc,fdi,gfcf,inflation,trade,domcred,gdp_growth
0,Brazil,1995,6596.335727,0.631586,20.286298,66.007034,16.984460,43.494525,NaN
1,Brazil,1996,6640.727007,1.475965,18.640654,15.757666,15.635591,40.778494,0.672969
2,Brazil,1997,6764.858421,2.150453,19.122901,6.926713,16.576209,40.852237,1.869244
3,Brazil,1998,6687.495236,3.340888,18.542349,3.195076,16.438585,29.532261,-1.143604
4,Brazil,1999,6621.636762,4.733770,17.016294,4.858447,20.982166,29.826910,-0.984800



Data types:
country        object
year            int64
gdp_pc        float64
fdi           float64
gfcf          float64
inflation     float64
trade         float64
domcred       float64
gdp_growth    float64
dtype: object

Number of missing rows: 0


# Descriptive Statistics & Correlation Matrix

In [54]:

# -------------------------
# 6) Descriptive stats & correlation (numeric only)
# -------------------------
numeric_df = df_final.select_dtypes(include=[np.number])
print("\nDescriptive statistics (numeric variables):")
display(numeric_df.describe())

# print("\nCorrelation matrix (numeric variables):")
# plt.figure(figsize=(8,6))
# sns.heatmap(numeric_df.corr(), annot=True, fmt=".2f", cmap='coolwarm', linewidths=0.5)
# plt.title("Correlation matrix (numeric variables)")
# plt.show()
numeric_df.corr()


Descriptive statistics (numeric variables):


,year,gdp_pc,fdi,gfcf,inflation,trade,domcred,gdp_growth
count,120.000000,120.000000,120.000000,120.000000,120.000000,120.000000,120.000000,120.000000
mean,2011.500000,6140.717139,2.239442,25.169138,5.904911,44.262542,78.513951,3.696654
std,6.951211,3095.918256,1.452731,9.418973,3.878271,11.693540,43.411918,3.974629
min,2000.000000,756.704110,-1.736810,13.051369,-0.731971,22.105976,16.837772,-7.827749
25%,2005.750000,3778.969933,1.127237,17.829198,3.658956,36.434811,47.066281,1.364196
50%,2011.500000,6140.348973,2.041694,21.158455,5.371229,46.093815,59.533968,3.889991
75%,2017.250000,8776.738975,3.206515,32.874006,7.040516,53.483109,118.528312,6.364832
max,2023.000000,12484.157884,9.660265,44.075543,21.477007,68.093907,189.606709,13.555366


,year,gdp_pc,fdi,gfcf,inflation,trade,domcred,gdp_growth
year,1.000000,0.391309,-0.148745,0.042574,-0.222934,-0.022910,0.249528,-0.290590
gdp_pc,0.391309,1.000000,0.059238,-0.227105,0.003540,-0.226036,0.211774,-0.270963
fdi,-0.148745,0.059238,1.000000,0.022743,-0.024582,-0.100758,-0.052322,0.171273
gfcf,0.042574,-0.227105,0.022743,1.000000,-0.387469,0.018286,0.393018,0.575152
inflation,-0.222934,0.003540,-0.024582,-0.387469,1.000000,0.286895,-0.559347,-0.116134
trade,-0.022910,-0.226036,-0.100758,0.018286,0.286895,1.000000,0.190436,0.226202
domcred,0.249528,0.211774,-0.052322,0.393018,-0.559347,0.190436,1.000000,0.078438
gdp_growth,-0.290590,-0.270963,0.171273,0.575152,-0.116134,0.226202,0.078438,1.000000


### Summary

- **GDP per capita tends to increase over time** but is negatively correlated with growth rates, which may indicate that wealthier countries grow slower.

- **Investment (gfcf) is a strong driver of GDP growth** in this dataset.

- **Inflation negatively affects both investment and domestic credit availability.**

- **Trade openness and domestic credit have moderate positive relations with growth.**

- Some variables like **FDI show weak relationships** with others, suggesting more complex dynamics or noisy data.
